In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.insert(0, config.root_path)

In [3]:
import tensorflow as tf

from tensorflow.python import keras
import json
import pandas as pd
import numpy as np
import os

from src.encoders.context_encoder_ldabert_2 import ContextEncoder
from src.dataset.ldabert_2 import LDABERT2Dataset
import transformers
import sentence_transformers

from utils.metrics import windowdiff, pk

In [4]:
tf.__version__, transformers.__version__, sentence_transformers.__version__

('2.5.0', '4.3.3', '0.4.1')

## LDA BERT 2 Dataset

In [5]:
dataset_type = "clinical"
dataset = LDABERT2Dataset(dataset_type=dataset_type,
                           pct_data=0.05,
                          max_seq_length=128,
                           max_segment_length=300,
                           augment_pct=0)

sentences, tokenized_sentences, labels = dataset.process()

# vectors_path = '../data/clinical_vectors/lda_bert_{}_{}.pkl'.format(dataset_type, len(sentences))
vectors_path = '../../data/lda_bert_2/{}/{}_{}_{}.pkl'.format(dataset_type, 
                                                            dataset.pct_data, 
                                                            dataset.augment_pct,
                                                             dataset.max_segment_length)

saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.get_saved_vectors(vectors_path)

if len(saved_vectors) == 0:
    saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.create_vectors(vectors_path)

In [6]:
left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)

In [7]:
lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

In [8]:
saved_tokenized_sentences['input_ids'].shape, saved_vectors.shape

(TensorShape([1453, 128]), (1453, 10))

## Model

In [9]:
model = ContextEncoder(final_dropout=0.5, 
                         dense_neurons=128, 
                         max_sentence_length=128, 
                       bert_trainable=False,
                         gamma=30)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [10]:
checkpoint_filepath = '../models/LDABERT2/simple/clinical-16122-1-pct-1-aug_OX2HS/no-finetune/checkpoint.ckpt'

In [11]:
# try:
model.load_weights(checkpoint_filepath)
# except:
#     print("No checkpoint available.")

## Inference

In [18]:
slice_size = 30

for i in range(0, saved_vectors.shape[0]//slice_size):
    start = i*slice_size
    end = start + slice_size
    print(start, end)
    print(len(saved_tokenized_sentences[start:end]))
    
    left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)
    lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors[start:end])
    
    logits = model([
        left_input, 
        mid_input, 
        right_input, 
        lda_left_input, 
        lda_mid_input, 
        lda_right_input
    ])

    logits_flattened = [float(p) for p in logits]

    predictions = [1 if float(logit) >= 0.5 else 0 for logit in logits_flattened]


    for i in range(1, 21):
        string_predictions = "".join([str(i) for i in predictions])
        string_ground_truth = "".join([str(i) for i in saved_labels])
        overall_windowdiff = windowdiff(string_predictions, string_ground_truth, i)
        overall_pk = pk(string_predictions, string_ground_truth, i)

        print("{},{},{},{}".format(overall_windowdiff, overall_pk, i, i))

    print(string_predictions)
    print(string_ground_truth)

0 30
30


KeyboardInterrupt: 

In [ ]:
[float(p) for p in predictions], saved_labels